In [1]:
# ! pip install statsmodels

In [2]:
import numpy as np
import QuantLib as ql
import pandas as pd
import statsmodels.api as sm

C:\Users\zhong\AppData\Local\Temp\ipykernel_11460\200167091.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Step 1. Construct 4 'Butterfly' portfolios

#### Read the data

In [3]:
data = pd.read_excel("Treasury Curve Data.xlsx")
data

,Date,USGG3M Index,USGG6M Index,USGG12M Index,USGG2YR Index,USGG3YR Index,USGG5YR Index,USGG7YR Index,USGG10YR Index,USGG30YR Index
0,2018-02-15,1.5879,1.8159,1.9769,2.1844,2.4076,2.6508,2.8362,2.9095,3.1637
1,2018-02-16,1.5901,1.8207,1.9714,2.1895,2.3835,2.6291,2.8039,2.8749,3.1316
2,2018-02-20,1.5952,1.8206,1.9869,2.2187,2.4055,2.6446,2.8140,2.8896,3.1530
3,2018-02-21,1.6390,1.8469,2.0025,2.2661,2.4358,2.6858,2.8672,2.9500,3.2204
4,2018-02-22,1.6338,1.8417,1.9998,2.2460,2.4085,2.6552,2.8345,2.9207,3.2062
...,...,...,...,...,...,...,...,...,...,...
1245,2023-02-09,4.7259,4.9019,4.8505,4.4820,4.1482,3.8586,3.7774,3.6579,3.7264
1246,2023-02-10,4.7345,4.8999,4.8680,4.5170,4.2015,3.9226,3.8480,3.7320,3.8158
1247,2023-02-13,4.7468,4.9232,4.8839,4.5175,4.2015,3.9104,3.8168,3.7016,3.7728
1248,2023-02-14,4.7142,4.9957,4.9513,4.6154,4.3169,4.0010,3.8848,3.7435,3.7746


#### Use the Feb-15-2023 yield curve to price the bonds

#### Use the QuantLib `ZeroCurve` function to bootstrap the yield curve. 


In [4]:
# old code
todaysDate = ql.Date(31, 1, 2022)
ql.Settings.instance().evaluationDate = todaysDate
Dates = [ql.Date(31, 1, 2022), ql.Date(30, 4, 2022), ql.Date(31, 7, 2022), ql.Date(31, 1, 2023), ql.Date(31, 1, 2024),ql.Date(31, 1, 2025), ql.Date(31, 1, 2027),ql.Date(31, 1, 2029), ql.Date(31, 1, 2032),ql.Date(31, 1, 2052)]
yields = [4.1207,4.7142,4.9584,4.9078,4.609,4.3056,3.9868,3.8796,3.7378,3.7737]
yields = [x/100 for x in yields]

dayCount = ql.Thirty360(ql.Thirty360.USA)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
spotCurve = ql.ZeroCurve(Dates, yields, dayCount, calendar, interpolation,
                        compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)

In [5]:
todaysDate = ql.Date(15, 2, 2023)
ql.Settings.instance().evaluationDate = todaysDate
Dates = [ql.Date(15,2,2023),ql.Date(15, 5, 2023), ql.Date(15, 8, 2023), ql.Date(15, 2, 2024), ql.Date(15, 2, 2025),ql.Date(15, 2, 2026), ql.Date(15, 2, 2028),ql.Date(15, 2, 2030), ql.Date(15, 2, 2033),ql.Date(15, 2, 2053)]
yields = [4.1207, 4.7142,4.9584,4.9078,4.609,4.3056,3.9868,3.8796,3.7378,3.7737]
yields = [x/100 for x in yields]

dayCount = ql.Thirty360(ql.Thirty360.USA)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
spotCurve = ql.ZeroCurve(Dates, yields, dayCount, calendar, interpolation,
                        compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)

### 2Y bond price and dollar duration

#### set up the variables to price the 2Y bond. 

In [6]:
issueDate = ql.Date(15, 2, 2023)
maturityDate = ql.Date(15, 2, 2025)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                        bussinessConvention , dateGeneration, monthEnd)
list(schedule)

[Date(15,2,2023),
 Date(15,8,2023),
 Date(15,2,2024),
 Date(15,8,2024),
 Date(15,2,2025)]

In [7]:
# Now lets build the coupon
dayCount = ql.Thirty360(ql.Thirty360.USA)
couponRate = 4.609/100
coupons = [couponRate]

# Now lets construct the FixedRateBond
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)

# create a bond engine with the term structure as input;
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

# Finally the price
fixedRateBond.NPV()

100.052993889883

In [8]:
P_2y = fixedRateBond.NPV()

In [9]:
# Calculate duration
duration_2y = ql.BondFunctions.duration(fixedRateBond, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)

# Assume a small change in yield (e.g., 1 basis point)
delta_y = 0.0001

# Calculate dollar duration
dollar_duration_2y = P_2y * duration_2y* delta_y
dollar_duration_2y

0.018566825572730785

### Calcualte 5Y Bond Price and dollar duration

In [10]:
issueDate = ql.Date(15, 2, 2023)
maturityDate = ql.Date(15, 2, 2028)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                        bussinessConvention , dateGeneration, monthEnd)
print(list(schedule))

dayCount = ql.Thirty360(ql.Thirty360.USA)
couponRate = 3.9868/100
coupons = [couponRate]

settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

# Finally the price
fixedRateBond.NPV()

[Date(15,2,2023), Date(15,8,2023), Date(15,2,2024), Date(15,8,2024), Date(15,2,2025), Date(15,8,2025), Date(15,2,2026), Date(15,8,2026), Date(15,2,2027), Date(15,8,2027), Date(15,2,2028)]


100.03856800016044

In [11]:
P_5y = fixedRateBond.NPV()

In [12]:
# Calculate duration
duration_5y = ql.BondFunctions.duration(fixedRateBond, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)

# Assume a small change in yield (e.g., 1 basis point)
delta_y = 0.0001

# Calculate dollar duration
dollar_duration_5y = P_5y * duration_5y* delta_y
dollar_duration_5y

0.04409296492448416

### Calculate 10Y Bond Price and dollar duration

In [13]:
issueDate = ql.Date(15, 2, 2023)
maturityDate = ql.Date(15, 2, 2033)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                        bussinessConvention , dateGeneration, monthEnd)
print(list(schedule))

dayCount = ql.Thirty360(ql.Thirty360.USA)
couponRate = 3.7378/100
coupons = [couponRate]

settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

# Finally the price
fixedRateBond.NPV()

[Date(15,2,2023), Date(15,8,2023), Date(15,2,2024), Date(15,8,2024), Date(15,2,2025), Date(15,8,2025), Date(15,2,2026), Date(15,8,2026), Date(15,2,2027), Date(15,8,2027), Date(15,2,2028), Date(15,8,2028), Date(15,2,2029), Date(15,8,2029), Date(15,2,2030), Date(15,8,2030), Date(15,2,2031), Date(15,8,2031), Date(15,2,2032), Date(15,8,2032), Date(15,2,2033)]


99.94512401871533

In [14]:
P_10y = fixedRateBond.NPV()

In [15]:
# Calculate duration
duration_10y = ql.BondFunctions.duration(fixedRateBond, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)

# Assume a small change in yield (e.g., 1 basis point)
delta_y = 0.0001

# Calculate dollar duration
dollar_duration_10y = P_10y * duration_10y* delta_y
dollar_duration_10y

0.08129766971114967


### Now, we have calcuated the dollar duration and duration of each bond and use them to construct the butterflies.


####  But, first we need to calculate the beta in strategy 3.

In [16]:
Y_10 = data.iloc[:,-2]
Y_5 = data.iloc[:,-4]
Y_2 = data.iloc[:,4]
Y10_5 = Y_10 - Y_5
Y5_2 = Y_5 - Y_2

Getting the changes on two sides

In [17]:
# x = Y10_5.diff().dropna()#[-65:] if 3 month
y = Y10_5.diff().dropna()#[-65:] if 3 month

In [18]:
# y = Y5_2.diff().dropna()#[-65:]
x = Y5_2.diff().dropna()#[-65:]

Fitting regression

In [19]:
x = sm.add_constant(x)
lm = sm.OLS(y,x).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     219.5
Date:                Wed, 21 Feb 2024   Prob (F-statistic):           7.11e-46
Time:                        15:14:24   Log-Likelihood:                 3273.0
No. Observations:                1249   AIC:                            -6542.
Df Residuals:                    1247   BIC:                            -6532.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001      0.000     -0.268      0.788      -0.001       0.001
0              0.3127      0.021     14.817      0.000       0.271       0.354
==============================================================================
Omnibus:                      167.108   Durbin-Watson:                   1.884
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1261.636
Skew:                          -0.357   Prob(JB):                    1.09e-274
Kurtosis:                       7.872   Cond. No.                         42.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
coefficients = lm.params
beta = coefficients[0]
beta

0.3127072051955262

<br>


### Constructing 4 butterflies portfolio

#### The cash-and $duration neutral weighting butterfly


In [21]:
# Matrix A1 represents the coefficients in the left-hand side of the equations
A1 = np.array([
    [dollar_duration_2y, dollar_duration_10y],
    [P_2y, P_10y]
])

# Vector b represents the right-hand side of the equations
b1 = np.array([
    -100 * dollar_duration_5y,  # For dollar duration neutrality
    -100 * P_5y   # For cash neutrality
])

# Solve the linear system A * x = b for vector x
x1 = np.linalg.lstsq(A1, b1, rcond=None)[0]

print(f"Quantity of 2Y Treasury (Q_2Y): {x1[0]}")
print(f"Quantity of 10Y Treasury (Q_10Y): {x1[1]}")

Quantity of 2Y Treasury (Q_2Y): -59.34662888111818
Quantity of 10Y Treasury (Q_10Y): -40.682814125349246


#### The fifty-fifty weighting regression

In [22]:
# Solve for Q_10Y (x)
Q_10Y = (- 100 * dollar_duration_5y) / (dollar_duration_10y + dollar_duration_10y)

# Then calculate Q_2Y using the fifty-fifty duration ratio
Q_2Y = (dollar_duration_10y / dollar_duration_2y) * Q_10Y

print(f"Quantity of 2Y Treasury (Q_2Y): {Q_2Y}")
print(f"Quantity of 10Y Treasury (Q_10Y): {Q_10Y}")

Quantity of 2Y Treasury (Q_2Y): -118.74125911228404
Quantity of 10Y Treasury (Q_10Y): -27.118221888244953


#### Regression Weighting Butterfly

In [23]:
# Calculate Q2y
Q2y = -100 * dollar_duration_5y / (dollar_duration_2y * (1 + 1/beta))

# Calculate Q10y using Q2y
Q10y = (Q2y * dollar_duration_2y) / (dollar_duration_10y * beta)

print(f"Quantity of 2-year bond (Q2y): {Q2y}")
print(f"Quantity of 10-year bond (Q10y): {Q10y}")

Quantity of 2-year bond (Q2y): -56.57201717403462
Quantity of 10-year bond (Q10y): -41.316482123225214


In [24]:
# beta = 1/beta
# # Calculate Q2y
# Q2y = -100 * dollar_duration_5y / (dollar_duration_2y * (1 + 1/beta))

# # Calculate Q10y using Q2y
# Q10y = (Q2y * dollar_duration_2y) / (dollar_duration_10y * beta)

# print(f"Quantity of 2-year bond (Q2y): {Q2y}")
# print(f"Quantity of 10-year bond (Q10y): {Q10y}")

#### Maturity-Weighting Butterfly

In [25]:
Q2y = (-75 * dollar_duration_5y) / (2 * dollar_duration_2y)
Q10y = (-125 * dollar_duration_5y) / (2 * dollar_duration_10y)

print(f"Quantity of 2-year bond (Q2y): {Q2y}")
print(f"Quantity of 10-year bond (Q10y): {Q10y}")

Quantity of 2-year bond (Q2y): -89.05594433421305
Quantity of 10-year bond (Q10y): -33.8977773603062


<br>


## Step 2. PCA curves

Calculate daily treasury shifts and standardize data

In [26]:
diff = data.diff().dropna().iloc[:,1:]
diff = (diff - diff.mean()) / diff.std()
covs = np.cov(np.array(np.array(diff)).T)
covs

array([[1.        , 0.66553498, 0.49571306, 0.38247303, 0.33597213,
        0.27970791, 0.24674973, 0.2265334 , 0.16711807],
       [0.66553498, 1.        , 0.71819365, 0.60324449, 0.54616478,
        0.4757455 , 0.42270933, 0.38025207, 0.28539135],
       [0.49571306, 0.71819365, 1.        , 0.7855207 , 0.73476908,
        0.65968051, 0.60106893, 0.54571962, 0.40662843],
       [0.38247303, 0.60324449, 0.7855207 , 1.        , 0.97067462,
        0.90311991, 0.83923056, 0.77011324, 0.58921292],
       [0.33597213, 0.54616478, 0.73476908, 0.97067462, 1.        ,
        0.9610309 , 0.9110838 , 0.84876369, 0.66735857],
       [0.27970791, 0.4757455 , 0.65968051, 0.90311991, 0.9610309 ,
        1.        , 0.98068918, 0.93999601, 0.78108127],
       [0.24674973, 0.42270933, 0.60106893, 0.83923056, 0.9110838 ,
        0.98068918, 1.        , 0.98203655, 0.86071605],
       [0.2265334 , 0.38025207, 0.54571962, 0.77011324, 0.84876369,
        0.93999601, 0.98203655, 1.        , 0.92941058],


In [27]:
eigs, eigvecs = np.linalg.eig(covs)
eigs

array([6.29967604e+00, 1.52639628e+00, 5.66764450e-01, 2.97666408e-01,
       2.12614593e-01, 7.09340708e-02, 1.49255046e-02, 7.33742844e-03,
       3.68522896e-03])

In [28]:
eigvecs

array([[-0.17548157, -0.58965003, -0.58750384,  0.49626573, -0.17301166,
        -0.0046569 ,  0.00450188,  0.00929525, -0.00415184],
       [-0.25376447, -0.52919131, -0.04047994, -0.56601801,  0.57549516,
        -0.04659178,  0.00391185, -0.01278774,  0.00091158],
       [-0.31140161, -0.32851626,  0.37517521, -0.31326071, -0.73237374,
        -0.13745989,  0.0306408 ,  0.00314687, -0.00145602],
       [-0.37128411, -0.06218633,  0.35716095,  0.26864765,  0.1217693 ,
         0.58567592, -0.5388091 ,  0.10160695,  0.00788672],
       [-0.3815586 ,  0.03591343,  0.26931473,  0.2854635 ,  0.1624173 ,
         0.14383093,  0.69500572, -0.40795732, -0.05074037],
       [-0.38464918,  0.1552955 ,  0.0811861 ,  0.17721871,  0.13760971,
        -0.36688007,  0.15991686,  0.70764504,  0.33175042],
       [-0.37831174,  0.22612582, -0.07593677,  0.06066915,  0.06295956,
        -0.41356487, -0.24750083, -0.08383751, -0.74364978],
       [-0.36636818,  0.27320959, -0.23265179, -0.07522209, -0

In [29]:
bp = 0.005 # 5 basis points

# Level: Scale PC1 such that the 10Y changes by +5bp
pc1 = eigvecs[:, 0]  # First principal component for level
scaling_factor_level = bp / pc1[-2]
level_shift = pc1 * scaling_factor_level

# Slope: Scale PC2 such that (10Y - 2Y) changes by +5bp
pc2 = eigvecs[:, 1]  # Second principal component for slope
scaling_factor_slope = bp / (pc2[-2] - pc2[4])
slope_shift = pc2 * scaling_factor_slope

# Curvature: Scale PC3 such that (2*5Y - (2Y + 10Y)) changes by +5bp
pc3 = eigvecs[:, 2]  # Third principal component for curvature
curvature_change_before_scaling = 2 * pc3[-4] - (pc3[4] + pc3[-2])
scaling_factor_curvature = bp / curvature_change_before_scaling  # Adjust indices as needed
curvature_shift = pc3 * scaling_factor_curvature

In [30]:
level_shift

array([0.00239488, 0.00346324, 0.00424985, 0.00506709, 0.00520731,
       0.00524949, 0.005163  , 0.005     , 0.00429624])

In [31]:
current_curve = [4.7142,4.9584,4.9078,4.609,4.3056,3.9868,3.8796,3.7378,3.7737]
current_curve = [x/100 for x in current_curve]

In [32]:
# Apply the shifts to the current curve to get the new scenarios
new_curve_level = current_curve + level_shift
print("new curve level: \n",new_curve_level)
print("\n")
new_curve_slope = current_curve + slope_shift
print("new_curve_slope: \n", new_curve_slope)
print("\n")
new_curve_curvature = current_curve + curvature_shift
print("new_curve_curvature: \n",new_curve_curvature)
print("\n")

new curve level: 
 [0.04953688 0.05304724 0.05332785 0.05115709 0.04826331 0.04511749
 0.043959   0.042378   0.04203324]


new_curve_slope: 
 [0.03471765 0.03843356 0.04215593 0.04477969 0.04381272 0.04314019
 0.04356063 0.04313472 0.04470298]


new_curve_curvature: 
 [0.02377444 0.04797394 0.06400034 0.06029583 0.05376781 0.04309712
 0.03577567 0.02812443 0.01801498]




## Step 4 Calculate individual bonds Prices under the 3 scenarios

In [46]:
# 2Y bond price
issueDate = ql.Date(15, 2, 2023)
maturityDate = ql.Date(15, 2, 2025)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                        bussinessConvention , dateGeneration, monthEnd)
# Now lets build the coupon
dayCount = ql.Thirty360(ql.Thirty360.USA)

couponRate_pc1 = 0.05115709
coupons_pc1 = [couponRate_pc1]

couponRate_pc2 = 0.04477969
coupons_pc2 = [couponRate_pc2]

couponRate_pc3 = 0.06029583
coupons_pc3 = [couponRate_pc3]

# Now lets construct the FixedRateBond
settlementDays = 0
faceValue = 100
# fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)
fixedRateBond_pc1 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc1, dayCount)
fixedRateBond_pc2 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc2, dayCount)
fixedRateBond_pc3 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc3, dayCount)

# create a bond engine with the term structure as input;
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc1.setPricingEngine(bondEngine)
# Finally the price
p_2y_pc1 = fixedRateBond_pc1.NPV()

bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc2.setPricingEngine(bondEngine)
# Finally the price
p_2y_pc2 = fixedRateBond_pc2.NPV()

bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc3.setPricingEngine(bondEngine)
# Finally the price
p_2y_pc3 = fixedRateBond_pc3.NPV()

p_2y_pc1, p_2y_pc2, p_2y_pc3

(101.00952937988589, 99.80564126383811, 102.73468704314217)

In [47]:
# 2Y bond dollar duration
duration_2y_pc1 = ql.BondFunctions.duration(fixedRateBond_pc1, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_2y_pc1 = p_2y_pc1 * duration_2y_pc1* delta_y

duration_2y_pc2 = ql.BondFunctions.duration(fixedRateBond_pc2, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_2y_pc2 = p_2y_pc2 * duration_2y_pc2* delta_y

duration_2y_pc3 = ql.BondFunctions.duration(fixedRateBond_pc3, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_2y_pc3 = p_2y_pc3 * duration_2y_pc3* delta_y

In [48]:
# 5Y bond price
issueDate = ql.Date(15, 2, 2023)
maturityDate = ql.Date(15, 2, 2028)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                        bussinessConvention , dateGeneration, monthEnd)
# Now lets build the coupon
dayCount = ql.Thirty360(ql.Thirty360.USA)

couponRate_pc1 = 0.04511749
coupons_pc1 = [couponRate_pc1]

couponRate_pc2 = 0.04314019
coupons_pc2 = [couponRate_pc2]

couponRate_pc3 = 0.04309712
coupons_pc3 = [couponRate_pc3]

# Now lets construct the FixedRateBond
settlementDays = 0
faceValue = 100
# fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)
fixedRateBond_pc1 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc1, dayCount)
fixedRateBond_pc2 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc2, dayCount)
fixedRateBond_pc3 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc3, dayCount)

# create a bond engine with the term structure as input;
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc1.setPricingEngine(bondEngine)
# Finally the price
p_5y_pc1 = fixedRateBond_pc1.NPV()

bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc2.setPricingEngine(bondEngine)
# Finally the price
p_5y_pc2 = fixedRateBond_pc2.NPV()

bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc3.setPricingEngine(bondEngine)
# Finally the price
p_5y_pc3 = fixedRateBond_pc3.NPV()

p_5y_pc1, p_5y_pc2, p_5y_pc3

(102.38149284945446, 101.49899468210695, 101.47977190555939)

In [49]:
# 5Y bond dollar duration
duration_5y_pc1 = ql.BondFunctions.duration(fixedRateBond_pc1, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_5y_pc1 = p_5y_pc1 * duration_5y_pc1* delta_y

duration_5y_pc2 = ql.BondFunctions.duration(fixedRateBond_pc2, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_5y_pc2 = p_5y_pc2 * duration_5y_pc2* delta_y

duration_5y_pc3 = ql.BondFunctions.duration(fixedRateBond_pc3, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_5y_pc3 = p_5y_pc3 * duration_5y_pc3* delta_y

In [50]:
# 10Y bond price
issueDate = ql.Date(15, 2, 2023)
maturityDate = ql.Date(15, 2, 2033)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                        bussinessConvention , dateGeneration, monthEnd)
# Now lets build the coupon
dayCount = ql.Thirty360(ql.Thirty360.USA)

couponRate_pc1 = 0.042378
coupons_pc1 = [couponRate_pc1]

couponRate_pc2 = 0.04313472
coupons_pc2 = [couponRate_pc2]

couponRate_pc3 = 0.02812443
coupons_pc3 = [couponRate_pc3]

# Now lets construct the FixedRateBond
settlementDays = 0
faceValue = 100
# fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)
fixedRateBond_pc1 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc1, dayCount)
fixedRateBond_pc2 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc2, dayCount)
fixedRateBond_pc3 = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons_pc3, dayCount)

# create a bond engine with the term structure as input;
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc1.setPricingEngine(bondEngine)
# Finally the price
p_10y_pc1 = fixedRateBond_pc1.NPV()

bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc2.setPricingEngine(bondEngine)
# Finally the price
p_10y_pc2 = fixedRateBond_pc2.NPV()

bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond_pc3.setPricingEngine(bondEngine)
# Finally the price
p_10y_pc3 = fixedRateBond_pc3.NPV()

p_10y_pc1, p_10y_pc2, p_10y_pc3

(104.04668758827637, 104.66743462514806, 92.35430289863875)

In [51]:
# 10Y bond dollar duration
duration_10y_pc1 = ql.BondFunctions.duration(fixedRateBond_pc1, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_10y_pc1 = p_10y_pc1 * duration_10y_pc1* delta_y

duration_10y_pc2 = ql.BondFunctions.duration(fixedRateBond_pc2, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_10y_pc2 = p_10y_pc2 * duration_10y_pc2* delta_y

duration_10y_pc3 = ql.BondFunctions.duration(fixedRateBond_pc3, ql.InterestRate(couponRate, dayCount, compounding, compoundingFrequency), ql.Duration.Modified)
dollar_duration_10y_pc3 = p_10y_pc3 * duration_10y_pc3* delta_y

## Step 5 Calculate the pnl for each of the butterfly portfolios under the 3 scenarios

The cash-and $duration neutral weighting butterfly

In [53]:
# Matrix A1 represents the coefficients in the left-hand side of the equations
A1 = np.array([
    [duration_2y_pc1, duration_10y_pc1],
    [p_2y_pc1, p_10y_pc1]
])

# Vector b represents the right-hand side of the equations
b1 = np.array([
    -100 * duration_5y_pc1,  # For dollar duration neutrality
    -100 * p_5y_pc1   # For cash neutrality
])

# Solve the linear system A * x = b for vector x
x1 = np.linalg.lstsq(A1, b1, rcond=None)[0]

q1_2y_pc1 = x1[0]
q1_10y_pc1 = x1[1]

print(f"Quantity of 2Y Treasury (Q_2Y): {x1[0]}")
print(f"Quantity of 10Y Treasury (Q_10Y): {x1[1]}")

Quantity of 2Y Treasury (Q_2Y): -59.20291915189695
Quantity of 10Y Treasury (Q_10Y): -40.92480387599278


The fifty-fifty weighting regression

In [55]:
# Solve for Q_10Y (x)
Q_10Y = (- 100 * duration_5y_pc1) / (duration_10y_pc1 + duration_10y_pc1)

# Then calculate Q_2Y using the fifty-fifty duration ratio
Q_2Y = (duration_10y_pc1 / duration_2y_pc1) * Q_10Y

q2_2y_pc1 = Q_2Y
q2_10y_pc1 = Q_10Y

print(f"Quantity of 2Y Treasury (Q_2Y): {Q_2Y}")
print(f"Quantity of 10Y Treasury (Q_10Y): {Q_10Y}")

Quantity of 2Y Treasury (Q_2Y): -117.35879000194579
Quantity of 10Y Treasury (Q_10Y): -27.364582768407942


Regression Weighting Butterfly

In [56]:
# Calculate Q2y
Q2y = -100 * duration_5y_pc1 / (duration_2y_pc1 * (1 + 1/beta))

# Calculate Q10y using Q2y
Q10y = (Q2y * duration_2y_pc1) / (duration_10y_pc1 * beta)

q3_2y_pc1 = Q2y
q3_10y_pc1 = Q10y
print(f"Quantity of 2-year bond (Q2y): {Q2y}")
print(f"Quantity of 10-year bond (Q10y): {Q10y}")

Quantity of 2-year bond (Q2y): -55.913366029206585
Quantity of 10-year bond (Q10y): -41.69182992232


Maturity-Weighting Butterfly

In [57]:
Q2y = (-75 * duration_5y_pc1) / (2 * duration_2y_pc1)
Q10y = (-125 * duration_5y_pc1) / (2 * duration_10y_pc1)

q4_2y_pc1 = Q2y
q4_10y_pc1 = Q10y

print(f"Quantity of 2-year bond (Q2y): {Q2y}")
print(f"Quantity of 10-year bond (Q10y): {Q10y}")

Quantity of 2-year bond (Q2y): -88.01909250145933
Quantity of 10-year bond (Q10y): -34.20572846050993


In [59]:
notional = 100000
pnl_pc1_1 = (q_2y_pc1 * (P_2y - p_2y_pc1) + q_10y_pc1 * (P_10y - p_10y_pc1) + 100 * (P_5y - p_5y_pc1)) * notional
pnl_pc1_1

-979706.1449604712